# GASTO MEDIO - CARACTERÍSTICAS CLIENTE 
# JOIN

# Proposito:

En este notebook nos dedicaremos a la unión del gasto medio con características de nuestro cliente. 

cargamos los ficheros y miramos los tipos de variables, creamos variables categoricas y hago una union simple de los datos con los ficheros que tienen fecha paies e isla. fin con eso creo que puedo hacer un predictor. ya luego veremos si se añaden características de soporte o que...


In [1]:
%load_ext watermark
%watermark

Last updated: 2023-02-10T12:20:41.859646+00:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 126 Stepping 5, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
import os
import re

# PREPARACION PREVIA

Primero necesitamos copiar todos nuestros archivos en una carpeta donde podramos manipularlos para así conservar los archivos originales

In [3]:
from distutils.dir_util import copy_tree

# Método que limpia la carpeta de datos volatiles ya que es 
# una carpeta activa en todos los programas que hemos generado.
def limpiaCarpetaVolatil(rutaCopia):
    print("Limpiando Carpeta Volatil...")
    for f in os.listdir(rutaCopia):
        os.remove(os.path.join(rutaCopia, f))
        
#Metodo que copia recursivamente
def copia(dir_origen,dir_destino):
    print("Copiando...")
    copy_tree(dir_origen, dir_destino)
    print("Copiado")
    print(os.listdir(dir_destino))
  


In [4]:
rutaBase= "../PredictorGastoTuristico/DatosGuardadosVirgenes/GM_JOIN_Carct_Cliente"
rutaCopia= "../PredictorGastoTuristico/DatosVolatiles/"

limpiaCarpetaVolatil(rutaCopia)
copia(rutaBase,rutaCopia)

Limpiando Carpeta Volatil...
Copiando...
Copiado
['gasto_medio_islas.csv', 'turistas_isla.csv.csv', 'turistas_motivos.csv.csv', 'turistas_motivos_isla.csv.csv', 'turistas_origen.csv.csv', 'turistas_paquete.csv.csv', 'turistas_paquete_isla.csv.csv', 'turistas_sexo_edad.csv.csv']


## Primero cargamos los archivos para tratarlos y finalmente unirlos:

In [5]:
ruta = rutaCopia
list_csv_names = os.listdir(ruta)
index = 0
for archivo in list_csv_names:
    print(str(index) + " " + archivo)
    index = index + 1

0 gasto_medio_islas.csv
1 turistas_isla.csv.csv
2 turistas_motivos.csv.csv
3 turistas_motivos_isla.csv.csv
4 turistas_origen.csv.csv
5 turistas_paquete.csv.csv
6 turistas_paquete_isla.csv.csv
7 turistas_sexo_edad.csv.csv


In [6]:
gasto_medio_islas = pd.read_csv(ruta+list_csv_names[0],keep_default_na=False, na_values=[""])
t_isla = pd.read_csv(ruta+list_csv_names[1],keep_default_na=False, na_values=[""])
t_motivos = pd.read_csv(ruta+list_csv_names[2],keep_default_na=False, na_values=[""])
t_motivos_isla = pd.read_csv(ruta+list_csv_names[3],keep_default_na=False, na_values=[""])
t_origen = pd.read_csv(ruta+list_csv_names[4],keep_default_na=False, na_values=[""])
t_paquete = pd.read_csv(ruta+list_csv_names[5],keep_default_na=False, na_values=[""])
t_paquete_isla = pd.read_csv(ruta+list_csv_names[6],keep_default_na=False, na_values=[""])
t_sexo_edad = pd.read_csv(ruta+list_csv_names[7],keep_default_na=False, na_values=[""])

In [7]:
list_all_csv = [gasto_medio_islas,t_isla,t_motivos,t_motivos_isla,t_origen,t_paquete,t_paquete_isla,t_sexo_edad]

# Empezamos trabajando con el archivo gasto_medio_islas como pivote

In [8]:
gasto_medio_islas.head(5)

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2019Q4,TOTAL,Gran Canaria,1254.62,510.15,413.23,331.24
1,2019Q3,TOTAL,Gran Canaria,1239.57,520.76,365.67,353.14
2,2019Q2,TOTAL,Gran Canaria,1124.73,474.48,340.77,309.48
3,2019Q1,TOTAL,Gran Canaria,1059.74,478.40,259.42,321.92
4,2018Q4,TOTAL,Gran Canaria,1205.52,528.63,341.18,335.71


In [9]:
gasto_medio_islas.dtypes

fecha                                                     object
pais                                                      object
isla                                                      object
total gasto por turista                                  float64
gasto por turista alojamiento                            float64
gasto por turista transporte nacional / internacional    float64
gasto por turista  otros                                 float64
dtype: object

Tal y como vemos los datos no mantienen los tipos establecidos en su tratamiento, asique volveremos a tratarlos nuevamente...

# 1. Astype 

### Columna `valor` tipo int y las columnas referentes al `Gasto` tipo float (CHECK)

## Trataremos las siguientes:

### Columnas `País, Islas, tipo_turista, motivo, paquete, sexo, edad` como Categóricas


hemos detectado que en la columna valor hay elementos que no son numeros, y son ".", ".." o "...", vamos a sustituir esto por "0"

In [10]:
i = 0
for archivo in list_all_csv:
    
#COLUMNAS COMUNES
    
    #FECHA
    archivo = archivo.astype({"fecha":'period[Q-DEC]'}) 
    
    #PAIS
    #archivo['pais'] = archivo['pais'].astype("category") -> ERR: SettingWithCopyWarning in Pandas
    archivo = archivo.astype({"pais":'category'}) 
    
#COLUMNAS NO COMUNES

    # ISLA
    if 'isla' in archivo.columns:
        archivo = archivo.astype({"isla":'category'})
    # TIPO_TURISTA
    if 'tipo_turista' in archivo.columns:
        archivo = archivo.astype({"tipo_turista":'category'})
    # MOTIVO
    if 'motivo' in archivo.columns:
        archivo = archivo.astype({"motivo":'category'})
    # PAQUETE
    if 'paquete' in archivo.columns:
        archivo = archivo.astype({"paquete":'category'})
    # EDAD
    if 'edad' in archivo.columns:
        archivo = archivo.astype({"edad":'category'})
    # SEXO
    if 'sexo' in archivo.columns:
        archivo = archivo.astype({"sexo":'category'})
    
    print("Se han modificado los dtypes del archivo: {} a: \n {}".format(
            list_csv_names[i], archivo.dtypes
    ))
     
    #Por alguna razon debemos guardar cambios porque de no hacerlo por alguna manera que
    # desconozco no se machaca la informacion existente y no se actualiza el dataframe...
    list_all_csv[i] = archivo
    i = i + 1

Se han modificado los dtypes del archivo: gasto_medio_islas.csv a: 
 fecha                                                    period[Q-DEC]
pais                                                          category
isla                                                          category
total gasto por turista                                        float64
gasto por turista alojamiento                                  float64
gasto por turista transporte nacional / internacional          float64
gasto por turista  otros                                       float64
dtype: object
Se han modificado los dtypes del archivo: turistas_isla.csv.csv a: 
 tipo_turista         category
fecha           period[Q-DEC]
pais                 category
isla                 category
valor                   int64
dtype: object
Se han modificado los dtypes del archivo: turistas_motivos.csv.csv a: 
 pais           category
motivo         category
fecha     period[Q-DEC]
valor             int64
dtype: object
Se ha

# Añadiendo características del cliente al gasto

# Al pivote `gasto_medio_islas` le iremos añadiendo mas datos.

# 2. Añadimos `t_turistas` y `t_origen` :

- T_turistas nos indica la cantidad de turistas que llegan a las islas de: `'Fuerteventura' 'Gran Canaria' 'La Palma' 'Lanzarote' 'Tenerife'`, 
- T_origen nos llegan la cantidad de turistas que llegan a `CANARIAS`

### En ambos casos `transformaremos las filas en columnas` para asi unirlo al gasto.

# 2.1 Tratamos t_isla

In [11]:
t_isla.head()

,tipo_turista,fecha,pais,isla,valor
0,Total,2010Q1,Alemania,Fuerteventura,143560
1,Total,2010Q1,Alemania,Gran Canaria,213576
2,Total,2010Q1,Alemania,La Palma,24677
3,Total,2010Q1,Alemania,Lanzarote,78239
4,Total,2010Q1,Alemania,Tenerife,166119


In [12]:
t_isla.tipo_turista.unique()

array(['Total', 'Turistas principales', 'Turistas secundarios'],
      dtype=object)

La columna tipo_turista tiene 3 valores, los cogeremos y crearemos columnas de su valor

In [13]:
#COLUMNA TOTAL TURISTAS
dft = t_isla[t_isla.tipo_turista.isin(["Total"])]              # Filtramos por Total
dft.rename(columns={'valor':'Total Turistas'}, inplace=True)   # Renombramos la columna valor
dft.drop('tipo_turista', axis=1, inplace=True)                 # Eliminamos la columna tipo_turista
dft.reset_index(drop=True, inplace=True)                       # Reiniciamos el indice

#COLUMNA TOTAL TURISTAS PRINCIPALES
dft2 = t_isla[t_isla.tipo_turista.isin(["Turistas principales"])]
dft2.rename(columns={'valor':'Total Turistas Principales'}, inplace=True)
dft2.drop('tipo_turista', axis=1, inplace=True)    
dft2.reset_index(drop=True, inplace=True)

#COLUMNA TOTAL TURISTAS SECUNDARIOS
dft3 = t_isla[t_isla.tipo_turista.isin(["Turistas secundarios"])]
dft3.rename(columns={'valor':'Total Turistas Secundarios'}, inplace=True)
dft3.drop('tipo_turista', axis=1, inplace=True)    
dft3.reset_index(drop=True, inplace=True)

#Aqui vemos como la division se ha hecho correctamente
print(dft.shape)
print(dft2.shape)
print(dft3.shape)

#Veamos un ejemplo de como quedan las tablas:
print(dft.head())

(1600, 4)
(1600, 4)
(1600, 4)
    fecha      pais           isla  Total Turistas
0  2010Q1  Alemania  Fuerteventura          143560
1  2010Q1  Alemania   Gran Canaria          213576
2  2010Q1  Alemania       La Palma           24677
3  2010Q1  Alemania      Lanzarote           78239
4  2010Q1  Alemania       Tenerife          166119


C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Aquí unimos los datos:

Hacemos uso de merge ya que con esa función hacemos uniones, en los parámetros indicamos los df a unir, luego ponemos las columnas que harán como clave para la unión y finalemnte indicamos left y right index a True que implica que (utilizamos el índice del DataFrame derch e izq como clave de unión)

Doc: https://stackoverflow.com/questions/51814612/understanding-the-left-index-and-right-index-arguments-in-pandas-merge

In [14]:
df = pd.merge(left=dft,right=dft2, on=['isla','fecha','pais'], left_index=True, right_index=True)
df.head(2)

,fecha,pais,isla,Total Turistas,Total Turistas Principales
0,2010Q1,Alemania,Fuerteventura,143560,139893
1,2010Q1,Alemania,Gran Canaria,213576,210319


In [15]:
df = pd.merge(left=df,right=dft3, on=['isla','fecha','pais'], left_index=True, right_index=True)
df.tail(2)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios
1598,2019Q4,TOTAL,Lanzarote,747450,707216,40234
1599,2019Q4,TOTAL,Tenerife,1498739,1456070,42669


In [16]:
df.shape

(1600, 6)

In [17]:
df.pais.unique()

array(['Alemania', 'España', 'Holanda', 'Otros países', 'Países Nórdicos',
       'RESIDENTES EN EL EXTRANJERO', 'Reino Unido', 'TOTAL'],
      dtype=object)

### Actualizamos t_isla con los datos transformamos

In [18]:
t_isla = df

# 2.2 Tratamos t_origen

In [19]:
t_origen.head()

,fecha,pais,valor
0,2010Q1,Alemania,620920
1,2010Q1,España,335230
2,2010Q1,Holanda,94439
3,2010Q1,Otros países,439882
4,2010Q1,Países Nórdicos,537221


In [20]:
gasto_medio_islas.isla.unique()

array(['Gran Canaria', 'Fuerteventura', 'La Palma', 'Lanzarote',
       'Tenerife', 'CANARIAS'], dtype=object)

In [21]:
t_origen.rename(columns={'valor':'Total Turistas'}, inplace=True)   # Renombramos la columna valor
t_origen['isla'] = "CANARIAS" # Añadimos la columna isla y referenciamos a CANARIAS en el mismo formato que el archivo gasto medio

In [22]:
t_origen.head()

,fecha,pais,Total Turistas,isla
0,2010Q1,Alemania,620920,CANARIAS
1,2010Q1,España,335230,CANARIAS
2,2010Q1,Holanda,94439,CANARIAS
3,2010Q1,Otros países,439882,CANARIAS
4,2010Q1,Países Nórdicos,537221,CANARIAS


# - NOTA IMPORTANTE:
### Debemos tener en cuenta que el estudio sera referente unica y exclusivamente a los datos de los paises que tiene gasto_medio

In [23]:
gasto_medio_islas.pais.unique()

array(['TOTAL', 'Alemania', 'España', 'Holanda', 'Países Nórdicos',
       'Reino Unido', 'Otros países'], dtype=object)

Vemos las shape de los 3 df a unir...

In [24]:
gasto_medio_islas.shape

(1680, 7)

In [25]:
t_isla.shape

(1600, 6)

In [26]:
t_origen.shape

(320, 4)

## 2.1 Unión `t_isla` y `t_origen` con`gasto_medio_islas`

### 2.1.1 Union  `t_isla` con `t_origen`

In [27]:
merged = pd.merge(t_isla,t_origen,on=['fecha','pais','isla','Total Turistas'],how="outer")
merged.shape

(1920, 6)

In [28]:
merged.isnull().sum()

fecha                           0
pais                            0
isla                            0
Total Turistas                  0
Total Turistas Principales    320
Total Turistas Secundarios    320
dtype: int64

In [29]:
merged.tail(3)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios
1917,2019Q4,RESIDENTES EN EL EXTRANJERO,CANARIAS,3504137,NaN,NaN
1918,2019Q4,Reino Unido,CANARIAS,1192902,NaN,NaN
1919,2019Q4,TOTAL,CANARIAS,3906658,NaN,NaN


### 2.1.2 Unión de lo anterior con `gasto_medio_islas`

In [30]:
merged = pd.merge(merged,gasto_medio_islas,on=['fecha','pais','isla'],how="outer")
merged.shape

(1920, 10)

In [31]:
merged.isnull().sum()

fecha                                                      0
pais                                                       0
isla                                                       0
Total Turistas                                             0
Total Turistas Principales                               320
Total Turistas Secundarios                               320
total gasto por turista                                  245
gasto por turista alojamiento                            250
gasto por turista transporte nacional / internacional    246
gasto por turista  otros                                 244
dtype: int64

Comprobamos que estan todos los rangos de fecha

In [32]:
merged.fecha.unique()

array(['2010Q1', '2010Q2', '2010Q3', '2010Q4', '2011Q1', '2011Q2',
       '2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4',
       '2013Q1', '2013Q2', '2013Q3', '2013Q4', '2014Q1', '2014Q2',
       '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3', '2015Q4',
       '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2',
       '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4',
       '2019Q1', '2019Q2', '2019Q3', '2019Q4'], dtype=object)

In [33]:
merged.tail(3)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
1917,2019Q4,RESIDENTES EN EL EXTRANJERO,CANARIAS,3504137,NaN,NaN,NaN,NaN,NaN,NaN
1918,2019Q4,Reino Unido,CANARIAS,1192902,NaN,NaN,1174.43,534.37,356.33,283.73
1919,2019Q4,TOTAL,CANARIAS,3906658,NaN,NaN,1174.79,492.21,373.99,308.59


In [34]:
merged.dtypes

fecha                                                     object
pais                                                      object
isla                                                      object
Total Turistas                                             int64
Total Turistas Principales                               float64
Total Turistas Secundarios                               float64
total gasto por turista                                  float64
gasto por turista alojamiento                            float64
gasto por turista transporte nacional / internacional    float64
gasto por turista  otros                                 float64
dtype: object

# 3. Añadimos `t_sexo_edad`:

t_sexo_edad nos indica la cantidad de turistas que llegan a las islas por sexo y edad, `transformaremos las filas en columnas` para asi unirlo al gasto.

# Unimos t_sexo_edad al gasto

In [35]:
t_sexo_edad.head(3)

,pais,sexo,edad,fecha,valor
0,Alemania,Hombres,De 15 a 24,2010Q1,16718
1,Alemania,Hombres,De 15 a 24,2010Q2,17735
2,Alemania,Hombres,De 15 a 24,2010Q3,33430


In [36]:
t_sexo_edad.sexo.unique()

array(['Hombres', 'Mujeres', 'Total'], dtype=object)

In [37]:
t_sexo_edad.edad.unique()

array(['De 15 a 24', 'De 25 a 44', 'De 45 a 64', 'Mayor de 64',
       'Menor de 15', 'TOTAL EDADES'], dtype=object)

La columna tipo_turista tiene 3 valores, los cogeremos y crearemos columnas de su valor

In [38]:
df_aux = t_sexo_edad

arr = []                  #creamos un array
#arr.append(dft)           #llenamos el array

#creo este bucle porque por cada sexo hay 6 rangos de edades asique filtro por sexo y por el rango de edad

for edad_value in t_sexo_edad.edad.unique():                   # BUCLE QUE RECORRE LOS 6 RANGOS DE EDAD
    
    #COLUMNA TOTAL TURISTAS POR SEXO - TOTAL 
##    dft = df_aux[df_aux.sexo.isin(["Total"])
##                 & df_aux.edad.isin([edad_value])]                                            # Filtramos por Total
##    
##    dft.rename(columns={'valor':'Total Sexo Turistas {}'.format(edad_value)}, inplace=True)   # Renombramos la columna valor
##    dft.drop(['sexo','edad'], axis=1, inplace=True)                                           # Eliminamos la columna tipo_turista
##    dft.reset_index(drop=True, inplace=True)                                                  # Reiniciamos el indice
##    arr.append(dft)                                                                           # lo añado al array
    
    #COLUMNA TOTAL TURISTAS POR SEXO - HOMBRES
    dft2 = df_aux[df_aux.sexo.isin(["Hombres"])
                 & df_aux.edad.isin([edad_value])]
                                    
    dft2.rename(columns={'valor':'Turistas Hombres {}'.format(edad_value)}, inplace=True)
    dft2.drop(['sexo','edad'], axis=1, inplace=True)    
    dft2.reset_index(drop=True, inplace=True)
    arr.append(dft2)
    
    #COLUMNA TOTAL TURISTAS POR SEXO - MUJERES
    dft3 = df_aux[df_aux.sexo.isin(["Mujeres"])
                 & df_aux.edad.isin([edad_value])]
    dft3.rename(columns={'valor':'Turistas Mujeres {}'.format(edad_value)}, inplace=True)
    dft3.drop(['sexo','edad'], axis=1, inplace=True)    
    dft3.reset_index(drop=True, inplace=True)
    arr.append(dft3)

    
#Aqui vemos como la division se ha hecho correctamente
#shape_df = t_sexo_edad.shape
#print(shape_df)
#for i in arr:
#    print(len(i.index))
print(arr)

C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[         pais   fecha  Turistas Hombres De 15 a 24
0    Alemania  2010Q1                        16718
1    Alemania  2010Q2                        17735
2    Alemania  2010Q3                        33430
3    Alemania  2010Q4                        23140
4    Alemania  2011Q1                        22464
..        ...     ...                          ...
315     TOTAL  2018Q4                        91794
316     TOTAL  2019Q1                       103494
317     TOTAL  2019Q2                       104493
318     TOTAL  2019Q3                       191107
319     TOTAL  2019Q4                        77709

[320 rows x 3 columns],          pais   fecha  Turistas Mujeres De 15 a 24
0    Alemania  2010Q1                        15374
1    Alemania  2010Q2                        17765
2    Alemania  2010Q3                        35957
3    Alemania  2010Q4                        22285
4    Alemania  2011Q1                        23362
..        ...     ...                          ...
315  

### Aquí unimos los datos:

Hacemos uso de merge ya que con esa función hacemos uniones, en los parámetros indicamos los df a unir, luego ponemos las columnas que harán como clave para la unión y finalemnte indicamos left y right index a True que implica que (utilizamos el índice del DataFrame derch e izq como clave de unión)

Doc: https://stackoverflow.com/questions/51814612/understanding-the-left-index-and-right-index-arguments-in-pandas-merge

In [39]:
inicio = 0
for elemento in arr:
    if inicio == 0:
        df = elemento
        inicio = inicio + 1
    else:
        df = pd.merge(df,elemento, on=['fecha','pais'], how="outer", left_index=True, right_index=True)
        #df = pd.merge(df,elemento, on=['fecha','pais'], left_index=True, right_index=True)
#df = df.dropna()
df.head()

,pais,fecha,Turistas Hombres De 15 a 24,Turistas Mujeres De 15 a 24,Turistas Hombres De 25 a 44,Turistas Mujeres De 25 a 44,Turistas Hombres De 45 a 64,Turistas Mujeres De 45 a 64,Turistas Hombres Mayor de 64,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES
0,Alemania,2010Q1,16718,15374,93932,100462,113098,121978,36697,40075,40365,42222,300809,320111
1,Alemania,2010Q2,17735,17765,87089,92636,70367,78805,21283,18286,33627,34834,230101,242324
2,Alemania,2010Q3,33430,35957,112512,112247,64539,67236,4812,5088,43101,47707,258395,268233
3,Alemania,2010Q4,23140,22285,121802,120086,109710,118944,24392,26468,35767,35112,314810,322895
4,Alemania,2011Q1,22464,23362,108888,111935,130845,128354,47431,47695,41687,39583,351314,350928


# estos datos referencia a canarias asique creamos una columna isla con valor CANARIAS

In [40]:
# Añadimos la columna isla y referenciamos a CANARIAS en el mismo formato que el archivo gasto medio
df['isla'] = "CANARIAS" 

In [41]:
df.shape

(320, 15)

In [42]:
df.tail()

,pais,fecha,Turistas Hombres De 15 a 24,Turistas Mujeres De 15 a 24,Turistas Hombres De 25 a 44,Turistas Mujeres De 25 a 44,Turistas Hombres De 45 a 64,Turistas Mujeres De 45 a 64,Turistas Hombres Mayor de 64,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES,isla
315,TOTAL,2018Q4,91794,117473,630672,620641,750282,785784,318008,334699,217192,189096,2007951,2047691,CANARIAS
316,TOTAL,2019Q1,103494,128255,594009,612850,710039,738660,351894,376212,244528,224362,2003963,2080338,CANARIAS
317,TOTAL,2019Q2,104493,139051,561694,578400,537529,578024,245986,272289,209426,207185,1659129,1774949,CANARIAS
318,TOTAL,2019Q3,191107,234664,642104,694042,556515,585733,110220,129821,273070,273398,1773017,1917657,CANARIAS
319,TOTAL,2019Q4,77709,94918,583178,562857,778898,787519,351938,369104,150486,150055,1942205,1964454,CANARIAS


# Segunda unión

In [43]:
merged = pd.merge(merged,df,on=['fecha','pais','isla'],how="outer")
merged.shape

(1920, 22)

In [44]:
merged.isnull().sum()

fecha                                                       0
pais                                                        0
isla                                                        0
Total Turistas                                              0
Total Turistas Principales                                320
Total Turistas Secundarios                                320
total gasto por turista                                   245
gasto por turista alojamiento                             250
gasto por turista transporte nacional / internacional     246
gasto por turista  otros                                  244
Turistas Hombres De 15 a 24                              1600
Turistas Mujeres De 15 a 24                              1600
Turistas Hombres De 25 a 44                              1600
Turistas Mujeres De 25 a 44                              1600
Turistas Hombres De 45 a 64                              1600
Turistas Mujeres De 45 a 64                              1600
Turistas

In [45]:
#merged2 = merged2.dropna()

In [46]:
merged.fecha.unique()

array(['2010Q1', '2010Q2', '2010Q3', '2010Q4', '2011Q1', '2011Q2',
       '2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4',
       '2013Q1', '2013Q2', '2013Q3', '2013Q4', '2014Q1', '2014Q2',
       '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3', '2015Q4',
       '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2',
       '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4',
       '2019Q1', '2019Q2', '2019Q3', '2019Q4'], dtype=object)

In [47]:
merged.tail()

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Hombres De 25 a 44,Turistas Mujeres De 25 a 44,Turistas Hombres De 45 a 64,Turistas Mujeres De 45 a 64,Turistas Hombres Mayor de 64,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES
1915,2019Q4,Otros países,CANARIAS,829163,NaN,NaN,1136.05,452.73,331.38,351.94,...,158701.0,146205.0,160629.0,160225.0,52625.0,54663.0,29869.0,29998.0,417362.0,411803.0
1916,2019Q4,Países Nórdicos,CANARIAS,575908,NaN,NaN,1406.66,490.14,557.86,358.66,...,63037.0,71739.0,94039.0,102284.0,64052.0,69275.0,40074.0,39393.0,275376.0,300531.0
1917,2019Q4,RESIDENTES EN EL EXTRANJERO,CANARIAS,3504137,NaN,NaN,NaN,NaN,NaN,NaN,...,493748.0,489040.0,698016.0,727298.0,325836.0,341497.0,137356.0,137520.0,1724456.0,1779681.0
1918,2019Q4,Reino Unido,CANARIAS,1192902,NaN,NaN,1174.43,534.37,356.33,283.73,...,153743.0,154631.0,252248.0,257076.0,118387.0,124423.0,36942.0,42440.0,585488.0,607414.0
1919,2019Q4,TOTAL,CANARIAS,3906658,NaN,NaN,1174.79,492.21,373.99,308.59,...,583178.0,562857.0,778898.0,787519.0,351938.0,369104.0,150486.0,150055.0,1942205.0,1964454.0


In [48]:
merged.isnull()

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Hombres De 25 a 44,Turistas Mujeres De 25 a 44,Turistas Hombres De 45 a 64,Turistas Mujeres De 45 a 64,Turistas Hombres Mayor de 64,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES
0,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
2,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1916,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1917,False,False,False,False,True,True,True,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1918,False,False,False,False,True,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [49]:
merged.head(10)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Hombres De 25 a 44,Turistas Mujeres De 25 a 44,Turistas Hombres De 45 a 64,Turistas Mujeres De 45 a 64,Turistas Hombres Mayor de 64,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES
0,2010Q1,Alemania,Fuerteventura,143560,139893.0,3668.0,1040.55,473.62,334.83,232.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010Q1,Alemania,Gran Canaria,213576,210319.0,3258.0,1195.45,498.81,350.78,345.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010Q1,Alemania,La Palma,24677,21531.0,3146.0,1052.89,380.01,290.08,382.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010Q1,Alemania,Lanzarote,78239,72847.0,5392.0,1012.47,406.05,293.55,312.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010Q1,Alemania,Tenerife,166119,165266.0,854.0,1242.21,476.28,362.34,403.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010Q1,España,Fuerteventura,26503,23225.0,3278.0,692.60,271.42,175.27,245.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2010Q1,España,Gran Canaria,91298,89850.0,1448.0,596.07,203.66,151.43,240.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2010Q1,España,La Palma,11051,9007.0,2045.0,629.53,239.70,185.56,204.28,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2010Q1,España,Lanzarote,70319,67496.0,2824.0,683.65,279.29,175.27,229.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2010Q1,España,Tenerife,150615,144212.0,6403.0,700.46,238.71,170.23,291.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# IMP MODIFICAR EL TEXTO M ARKDOWN

# 4. Añadimos `t_motivos` y `t_motivos_isla` :

- t_motivos_isla nos indica la cantidad de turistas que llegan a las islas de: `'Fuerteventura' 'Gran Canaria' 'La Palma' 'Lanzarote' 'Tenerife'` en funcion de los motivos. 

- t_motivos nos llegan la cantidad de turistas que llegan a `CANARIAS` en funcion de los motivos.

Em ambos casos los motivos pueden ser: `'Ocio o vacaciones', 'Otros motivos', 'Personal', 'TOTAL MOTIVOS','Trabajo o negocios'`


En ambos casos `transformaremos las filas en columnas` para asi unirlo al gasto.

# 4.1 Tratamos t_motivos

In [50]:
t_motivos.head()

,pais,motivo,fecha,valor
0,Alemania,Ocio o vacaciones,2010Q1,604627
1,Alemania,Ocio o vacaciones,2010Q2,460804
2,Alemania,Ocio o vacaciones,2010Q3,521401
3,Alemania,Ocio o vacaciones,2010Q4,626062
4,Alemania,Ocio o vacaciones,2011Q1,695473


In [51]:
gasto_medio_islas.isla.unique()

array(['Gran Canaria', 'Fuerteventura', 'La Palma', 'Lanzarote',
       'Tenerife', 'CANARIAS'], dtype=object)

In [52]:
# Añadimos la columna isla y referenciamos a CANARIAS en el mismo formato que el archivo gasto medio
t_motivos['isla'] = "CANARIAS" 

In [53]:
t_motivos.head(2)

,pais,motivo,fecha,valor,isla
0,Alemania,Ocio o vacaciones,2010Q1,604627,CANARIAS
1,Alemania,Ocio o vacaciones,2010Q2,460804,CANARIAS


In [54]:
t_motivos_isla.head(2)

,motivo,fecha,pais,isla,valor
0,Ocio o vacaciones,2010Q1,España,Fuerteventura,22685
1,Ocio o vacaciones,2010Q1,España,Gran Canaria,57676


Vemos las shape de los 2 df a unir...

In [55]:
print(" t_motivos:{} \n t_motivos_isla:{}".format(t_motivos.shape,t_motivos_isla.shape))

 t_motivos:(1600, 5) 
 t_motivos_isla:(3000, 5)


## 4.2 Unión `t_motivos` y `t_motivos_isla` y con gasto_medio_islas y todo lo demas...

### 4.2.1 Union  `t_motivos` con `t_motivos_isla`

In [56]:
merged_motivos = pd.merge(t_motivos,t_motivos_isla,on=['fecha','pais','isla','valor',"motivo"],how="outer")
merged_motivos.shape

(4600, 5)

In [57]:
merged_motivos.isnull().sum()

pais      0
motivo    0
fecha     0
valor     0
isla      0
dtype: int64

In [58]:
merged_motivos.tail(3)

,pais,motivo,fecha,valor,isla
4597,TOTAL,Trabajo o negocios,2019Q4,1798,La Palma
4598,TOTAL,Trabajo o negocios,2019Q4,7081,Lanzarote
4599,TOTAL,Trabajo o negocios,2019Q4,35971,Tenerife


In [59]:
merged_motivos.motivo.unique()

array(['Ocio o vacaciones', 'Otros motivos', 'Personal', 'TOTAL MOTIVOS',
       'Trabajo o negocios'], dtype=object)

In [60]:
df_aux = merged_motivos

arr = []                  #creamos un array

#creo este bucle porque por cada motivo hay 5, asique creo 5 columnas

for motivo_value in merged_motivos.motivo.unique():                   # BUCLE QUE RECORRE LOS 6 RANGOS DE EDAD
    
    #COLUMNA TOTAL TURISTAS POR SEXO - TOTAL 
    dft = df_aux[df_aux.motivo.isin([motivo_value])]                                            # Filtramos por Total
    
    dft.rename(columns={'valor':'Total Turistas por motivo: {}'.format(motivo_value)}, inplace=True)   # Renombramos la columna valor
    dft.drop(['motivo'], axis=1, inplace=True)                                           # Eliminamos la columna tipo_turista
    dft.reset_index(drop=True, inplace=True)                                                  # Reiniciamos el indice
    arr.append(dft)                                                                           # lo añado al array
    
print(arr)

[         pais   fecha  Total Turistas por motivo: Ocio o vacaciones  \
0    Alemania  2010Q1                                        604627   
1    Alemania  2010Q2                                        460804   
2    Alemania  2010Q3                                        521401   
3    Alemania  2010Q4                                        626062   
4    Alemania  2011Q1                                        695473   
..        ...     ...                                           ...   
915     TOTAL  2019Q4                                        493268   
916     TOTAL  2019Q4                                       1088282   
917     TOTAL  2019Q4                                         86625   
918     TOTAL  2019Q4                                        735247   
919     TOTAL  2019Q4                                       1430686   

              isla  
0         CANARIAS  
1         CANARIAS  
2         CANARIAS  
3         CANARIAS  
4         CANARIAS  
..             ...  

C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Aquí unimos los datos:

Hacemos uso de merge ya que con esa función hacemos uniones, en los parámetros indicamos los df a unir, luego ponemos las columnas que harán como clave para la unión y finalemnte indicamos left y right index a True que implica que (utilizamos el índice del DataFrame derch e izq como clave de unión)

Doc: https://stackoverflow.com/questions/51814612/understanding-the-left-index-and-right-index-arguments-in-pandas-merge

In [61]:
inicio = 0
for elemento in arr:
    if inicio == 0:
        df = elemento
        inicio = inicio + 1
    else:
        df = pd.merge(df,elemento, on=['fecha','pais','isla'], how="outer", left_index=True, right_index=True)
df.head()

,pais,fecha,Total Turistas por motivo: Ocio o vacaciones,isla,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios
0,Alemania,2010Q1,604627,CANARIAS,3633,8526,620920,4134
1,Alemania,2010Q2,460804,CANARIAS,2724,4190,472425,4709
2,Alemania,2010Q3,521401,CANARIAS,565,2576,526628,2086
3,Alemania,2010Q4,626062,CANARIAS,1488,3604,637703,6551
4,Alemania,2011Q1,695473,CANARIAS,386,2608,702242,3774


In [62]:
df.shape

(920, 8)

In [63]:
df.tail()

,pais,fecha,Total Turistas por motivo: Ocio o vacaciones,isla,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios
915,TOTAL,2019Q4,493268,Fuerteventura,903,2470,503541,6901
916,TOTAL,2019Q4,1088282,Gran Canaria,3866,15981,1177156,69029
917,TOTAL,2019Q4,86625,La Palma,722,492,89635,1798
918,TOTAL,2019Q4,735247,Lanzarote,1645,3479,747450,7081
919,TOTAL,2019Q4,1430686,Tenerife,8513,23568,1498739,35971


### 4.2.2 Unión de lo anterior con `gasto_medio_islas`

In [64]:
merged = pd.merge(merged,df,on=['fecha','pais','isla'],how="outer")
merged.shape

(2120, 27)

In [65]:
merged.isnull().sum()

fecha                                                       0
pais                                                        0
isla                                                        0
Total Turistas                                            200
Total Turistas Principales                                520
Total Turistas Secundarios                                520
total gasto por turista                                   445
gasto por turista alojamiento                             450
gasto por turista transporte nacional / internacional     446
gasto por turista  otros                                  444
Turistas Hombres De 15 a 24                              1800
Turistas Mujeres De 15 a 24                              1800
Turistas Hombres De 25 a 44                              1800
Turistas Mujeres De 25 a 44                              1800
Turistas Hombres De 45 a 64                              1800
Turistas Mujeres De 45 a 64                              1800
Turistas

Visualizamos la unión

In [66]:
merged.head(5)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Mujeres Mayor de 64,Turistas Hombres Menor de 15,Turistas Mujeres Menor de 15,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES,Total Turistas por motivo: Ocio o vacaciones,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios
0,2010Q1,Alemania,Fuerteventura,143560.0,139893.0,3668.0,1040.55,473.62,334.83,232.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010Q1,Alemania,Gran Canaria,213576.0,210319.0,3258.0,1195.45,498.81,350.78,345.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010Q1,Alemania,La Palma,24677.0,21531.0,3146.0,1052.89,380.01,290.08,382.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010Q1,Alemania,Lanzarote,78239.0,72847.0,5392.0,1012.47,406.05,293.55,312.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010Q1,Alemania,Tenerife,166119.0,165266.0,854.0,1242.21,476.28,362.34,403.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 5. Añadimos `t_paquete` y `t_paquete_isla` :

- t_paquete_isla nos indica la cantidad de turistas que llegan a las islas de: `'Fuerteventura' 'Gran Canaria' 'La Palma' 'Lanzarote' 'Tenerife'` que contratan o no un paquete

- t_paquete nos llegan la cantidad de turistas que llegan a `CANARIAS` que contratan o no un paquete

En ambos casos `transformaremos las filas en columnas` para asi unirlo al gasto.

# 5.1 Tratamos t_paquete

In [67]:
t_paquete.head()

,pais,paquete,fecha,valor
0,Alemania,No contrataron un paquete turístico,2010Q1,154831
1,Alemania,No contrataron un paquete turístico,2010Q2,118358
2,Alemania,No contrataron un paquete turístico,2010Q3,130379
3,Alemania,No contrataron un paquete turístico,2010Q4,179361
4,Alemania,No contrataron un paquete turístico,2011Q1,199214


In [68]:
gasto_medio_islas.isla.unique()

array(['Gran Canaria', 'Fuerteventura', 'La Palma', 'Lanzarote',
       'Tenerife', 'CANARIAS'], dtype=object)

In [69]:
# Añadimos la columna isla y referenciamos a CANARIAS en el mismo formato que el archivo gasto medio
t_paquete['isla'] = "CANARIAS" 

In [70]:
t_paquete.head(2)

,pais,paquete,fecha,valor,isla
0,Alemania,No contrataron un paquete turístico,2010Q1,154831,CANARIAS
1,Alemania,No contrataron un paquete turístico,2010Q2,118358,CANARIAS


In [71]:
t_paquete_isla.head(2)

,paquete,fecha,pais,isla,valor
0,No contrataron un paquete turístico,2010Q1,España,Fuerteventura,19074
1,No contrataron un paquete turístico,2010Q1,España,Gran Canaria,77396


Vemos las shape de los 2 df a unir...

In [72]:
print(" t_paquete:{} \n t_paquete_isla:{}".format(t_paquete.shape,t_paquete_isla.shape))

 t_paquete:(960, 5) 
 t_paquete_isla:(1800, 5)


## 5.2 Unión `t_paquete` y `t_paquete_isla` y con gasto_medio_islas y todo lo demas...

### 5.2.1 Union  `t_paquete` con `t_paquete_isla`

In [73]:
merged_paquete = pd.merge(t_paquete,t_paquete_isla,on=['fecha','pais','isla','valor',"paquete"],how="outer")
merged_paquete.shape

(2760, 5)

In [74]:
merged_paquete.isnull().sum()

pais       0
paquete    0
fecha      0
valor      0
isla       0
dtype: int64

In [75]:
merged_paquete.tail(3)

,pais,paquete,fecha,valor,isla
2757,TOTAL,Total,2019Q4,89635,La Palma
2758,TOTAL,Total,2019Q4,747450,Lanzarote
2759,TOTAL,Total,2019Q4,1498739,Tenerife


In [76]:
merged_paquete.paquete.unique()

array(['No contrataron un paquete turístico',
       'Sí contrataron un paquete turístico', 'Total'], dtype=object)

## IMP! ESTE CODIGO QUE ESTAMOS HACIENDO AQUI CLARAMENTE SIGUE UN PATRON POR LO QUE PUEDE HACERSE CALARAMENRTE UN MÉTODO, PERO REQUIERE TIEMPO Y PARA EL CASO QUE NOS OCUPA ESTO RESULTA MAS RAPIDO EN TIEMPO/BENEFICIO


In [77]:
df_aux = merged_paquete

arr = []                  #creamos un array

#creo este bucle porque por cada motivo hay 5, asique creo 5 columnas

for paquete_value in merged_paquete.paquete.unique():                   # BUCLE QUE RECORRE LOS 6 RANGOS DE EDAD
    
    #COLUMNA TOTAL TURISTAS POR SEXO - TOTAL 
    dft = df_aux[df_aux.paquete.isin([paquete_value])]                                            # Filtramos por Total
    
    dft.rename(columns={'valor':'Total Turistas: {}'.format(paquete_value)}, inplace=True)   # Renombramos la columna valor
    dft.drop(['paquete'], axis=1, inplace=True)                                           # Eliminamos la columna tipo_turista
    dft.reset_index(drop=True, inplace=True)                                                  # Reiniciamos el indice
    arr.append(dft)                                                                           # lo añado al array
    
print(arr)

[         pais   fecha  Total Turistas: No contrataron un paquete turístico  \
0    Alemania  2010Q1                                             154831     
1    Alemania  2010Q2                                             118358     
2    Alemania  2010Q3                                             130379     
3    Alemania  2010Q4                                             179361     
4    Alemania  2011Q1                                             199214     
..        ...     ...                                                ...     
915     TOTAL  2019Q4                                             165102     
916     TOTAL  2019Q4                                             479759     
917     TOTAL  2019Q4                                              41971     
918     TOTAL  2019Q4                                             309498     
919     TOTAL  2019Q4                                             714343     

              isla  
0         CANARIAS  
1         CANARIAS  

C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
C:\Users\angel\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


### Aquí unimos los datos:

Hacemos uso de merge ya que con esa función hacemos uniones, en los parámetros indicamos los df a unir, luego ponemos las columnas que harán como clave para la unión y finalemnte indicamos left y right index a True que implica que (utilizamos el índice del DataFrame derch e izq como clave de unión)

Doc: https://stackoverflow.com/questions/51814612/understanding-the-left-index-and-right-index-arguments-in-pandas-merge

In [78]:
inicio = 0
for elemento in arr:
    if inicio == 0:
        df = elemento
        inicio = inicio + 1
    else:
        df = pd.merge(df,elemento, on=['fecha','pais','isla'], how="outer", left_index=True, right_index=True)
df.head()

,pais,fecha,Total Turistas: No contrataron un paquete turístico,isla,Total Turistas: Sí contrataron un paquete turístico,Total Turistas: Total
0,Alemania,2010Q1,154831,CANARIAS,466089,620920
1,Alemania,2010Q2,118358,CANARIAS,354067,472425
2,Alemania,2010Q3,130379,CANARIAS,396249,526628
3,Alemania,2010Q4,179361,CANARIAS,458343,637703
4,Alemania,2011Q1,199214,CANARIAS,503028,702242


In [79]:
df.shape

(920, 6)

In [80]:
df.tail()

,pais,fecha,Total Turistas: No contrataron un paquete turístico,isla,Total Turistas: Sí contrataron un paquete turístico,Total Turistas: Total
915,TOTAL,2019Q4,165102,Fuerteventura,338439,503541
916,TOTAL,2019Q4,479759,Gran Canaria,697396,1177156
917,TOTAL,2019Q4,41971,La Palma,47664,89635
918,TOTAL,2019Q4,309498,Lanzarote,437953,747450
919,TOTAL,2019Q4,714343,Tenerife,784396,1498739


### 5.2.2 Unión de lo anterior con `gasto_medio_islas`

In [81]:
merged = pd.merge(merged,df,on=['fecha','pais','isla'],how="outer")
merged.shape

(2120, 30)

In [82]:
merged.isnull().sum()

fecha                                                       0
pais                                                        0
isla                                                        0
Total Turistas                                            200
Total Turistas Principales                                520
Total Turistas Secundarios                                520
total gasto por turista                                   445
gasto por turista alojamiento                             450
gasto por turista transporte nacional / internacional     446
gasto por turista  otros                                  444
Turistas Hombres De 15 a 24                              1800
Turistas Mujeres De 15 a 24                              1800
Turistas Hombres De 25 a 44                              1800
Turistas Mujeres De 25 a 44                              1800
Turistas Hombres De 45 a 64                              1800
Turistas Mujeres De 45 a 64                              1800
Turistas

Visualizamos la unión

In [83]:
merged.head(10)

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES,Total Turistas por motivo: Ocio o vacaciones,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios,Total Turistas: No contrataron un paquete turístico,Total Turistas: Sí contrataron un paquete turístico,Total Turistas: Total
0,2010Q1,Alemania,Fuerteventura,143560.0,139893.0,3668.0,1040.55,473.62,334.83,232.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010Q1,Alemania,Gran Canaria,213576.0,210319.0,3258.0,1195.45,498.81,350.78,345.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010Q1,Alemania,La Palma,24677.0,21531.0,3146.0,1052.89,380.01,290.08,382.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010Q1,Alemania,Lanzarote,78239.0,72847.0,5392.0,1012.47,406.05,293.55,312.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010Q1,Alemania,Tenerife,166119.0,165266.0,854.0,1242.21,476.28,362.34,403.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2010Q1,España,Fuerteventura,26503.0,23225.0,3278.0,692.60,271.42,175.27,245.91,...,NaN,NaN,22685.0,0.0,731.0,26503.0,3088.0,19074.0,7428.0,26503.0
6,2010Q1,España,Gran Canaria,91298.0,89850.0,1448.0,596.07,203.66,151.43,240.97,...,NaN,NaN,57676.0,1682.0,3700.0,91298.0,28240.0,77396.0,13902.0,91298.0
7,2010Q1,España,La Palma,11051.0,9007.0,2045.0,629.53,239.70,185.56,204.28,...,NaN,NaN,9072.0,0.0,166.0,11051.0,1813.0,5281.0,5770.0,11051.0
8,2010Q1,España,Lanzarote,70319.0,67496.0,2824.0,683.65,279.29,175.27,229.09,...,NaN,NaN,66992.0,27.0,357.0,70319.0,2944.0,35716.0,34604.0,70319.0
9,2010Q1,España,Tenerife,150615.0,144212.0,6403.0,700.46,238.71,170.23,291.52,...,NaN,NaN,123170.0,2005.0,2343.0,150615.0,23098.0,72316.0,78298.0,150615.0


# OPERACIONES DE FILTRADO NECESARIAS...
Hemos visto que existen muchos elementos que son nulos debido a que hay algunos archivos que hemos unido que hacen referencia solo a `España, Total y extranjeros`,  esto implica que para los valores que se refieren a otros países pues no tengamos datos Es por ello por lo que hemos decidido hacer una serie de filtrado para quedarnos con los datos que son más útiles. Para ello filtramos primero por países quedándonos solo con los países de los que se hacen referencia a gasto_medio_islas ya que éstos son los únicos países que nos interesan ya que nuestro objetivo principal es el de predecir el gasto y finalmente también filtramos por la columna isla esta columna hace referencia a diferentes islas `LNZ,FTV,TNF… CANARIAS` nos quedamos con las filas referentes a datos de CANARIAS Ya que son las únicas filas que convergen en todos los datos que hemos ido uniendo.

# FILTRAMOS SOLO POR LOS PAISES QUE HAY EN GASTO_MEDIO_ISLAS

In [84]:
gasto_medio_islas.pais.unique()

array(['TOTAL', 'Alemania', 'España', 'Holanda', 'Países Nórdicos',
       'Reino Unido', 'Otros países'], dtype=object)

In [85]:
filtroPAISES = merged[merged.pais.isin(['TOTAL', 'Alemania', 'España', 'Holanda', 'Países Nórdicos',
       'Reino Unido', 'Otros países'])]                              
filtroPAISES.reset_index(drop=True, inplace=True)
filtroPAISES.shape

(1680, 30)

In [86]:
filtroPAISES.head()

,fecha,pais,isla,Total Turistas,Total Turistas Principales,Total Turistas Secundarios,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,...,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES,Total Turistas por motivo: Ocio o vacaciones,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios,Total Turistas: No contrataron un paquete turístico,Total Turistas: Sí contrataron un paquete turístico,Total Turistas: Total
0,2010Q1,Alemania,Fuerteventura,143560.0,139893.0,3668.0,1040.55,473.62,334.83,232.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010Q1,Alemania,Gran Canaria,213576.0,210319.0,3258.0,1195.45,498.81,350.78,345.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010Q1,Alemania,La Palma,24677.0,21531.0,3146.0,1052.89,380.01,290.08,382.81,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010Q1,Alemania,Lanzarote,78239.0,72847.0,5392.0,1012.47,406.05,293.55,312.86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010Q1,Alemania,Tenerife,166119.0,165266.0,854.0,1242.21,476.28,362.34,403.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
filtroPAISES.isnull().sum()

fecha                                                       0
pais                                                        0
isla                                                        0
Total Turistas                                              0
Total Turistas Principales                                280
Total Turistas Secundarios                                280
total gasto por turista                                     5
gasto por turista alojamiento                              10
gasto por turista transporte nacional / internacional       6
gasto por turista  otros                                    4
Turistas Hombres De 15 a 24                              1400
Turistas Mujeres De 15 a 24                              1400
Turistas Hombres De 25 a 44                              1400
Turistas Mujeres De 25 a 44                              1400
Turistas Hombres De 45 a 64                              1400
Turistas Mujeres De 45 a 64                              1400
Turistas

# GUARDAMOS...

### AQUI YA TENEMOS EL ARCHIVO PARA PODER AÑADIR MAS COSAS, DONDE TENEMOS TODO FILTRADO POR LOS PAISES DE GM QUE AL FINAL ES LO UNICO QUE PODREMOS PREDECIR

In [88]:
rutaDatosTratados = "../PredictorGastoTuristico/DatosGuardados/"
# GMI_join_cc.csv -> gasto_medio_islas_join_caracteristicas_cliente
filtroPAISES.to_csv(rutaDatosTratados+"GMI_join_cc.csv", index = False, encoding='utf-8')

# HAY MUCHOS NULOS PORQUE HAY ARCHIVOS COMO TOTALMOTIVOS Y DEMAS QUE SOLO TIEENEN PAISES COMO ESPAÑA,TOTAL Y EXTRANJEROS

# UNA OPCION ES COGER TODOS LOS PAISES Q NO SON ESPAÑA Y PONERLOS COMO EXTRANJEROS... Y ASI TENGO MAS DATOS

# OTRO FILTRO MAS

## VOY A FILTRAR SOLO POR ISLA -> CANARIAS DESDE EL FILTRO ANTERIOR

VOY A FILTRAR SOLO POR ISLA -> CANARIAS desde el filtor de paises

In [89]:
filtro_canarias = filtroPAISES[filtroPAISES.isla.isin(["CANARIAS"])]                            
#filtro_canarias.drop(['sexo','edad'], axis=1, inplace=True)    
filtro_canarias.reset_index(drop=True, inplace=True)
filtro_canarias.shape

(280, 30)

Elimino columnas y filas nulas

In [90]:
filtro_canarias = filtro_canarias.dropna(1) #elimino columnas nulas
filtro_canarias = filtro_canarias.dropna() # elimino filas nulas
filtro_canarias.shape

(280, 28)

In [91]:
filtro_canarias.head()

,fecha,pais,isla,Total Turistas,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros,Turistas Hombres De 15 a 24,Turistas Mujeres De 15 a 24,...,Turistas Hombres TOTAL EDADES,Turistas Mujeres TOTAL EDADES,Total Turistas por motivo: Ocio o vacaciones,Total Turistas por motivo: Otros motivos,Total Turistas por motivo: Personal,Total Turistas por motivo: TOTAL MOTIVOS,Total Turistas por motivo: Trabajo o negocios,Total Turistas: No contrataron un paquete turístico,Total Turistas: Sí contrataron un paquete turístico,Total Turistas: Total
0,2010Q1,Alemania,CANARIAS,620920.0,1147.19,472.84,341.47,332.88,16718.0,15374.0,...,300809.0,320111.0,604627.0,3633.0,8526.0,620920.0,4134.0,154831.0,466089.0,620920.0
1,2010Q1,España,CANARIAS,335230.0,666.96,238.65,166.80,261.51,10179.0,8403.0,...,180231.0,154998.0,270814.0,3700.0,7096.0,335230.0,53619.0,201097.0,134132.0,335230.0
2,2010Q1,Holanda,CANARIAS,94439.0,1107.77,455.40,337.60,314.77,2861.0,2899.0,...,46656.0,47784.0,90723.0,2245.0,511.0,94439.0,961.0,21215.0,73224.0,94439.0
3,2010Q1,Otros países,CANARIAS,439882.0,1023.57,417.21,291.47,314.90,11874.0,12365.0,...,215703.0,224178.0,420568.0,2292.0,5573.0,439881.0,11445.0,185037.0,254843.0,439881.0
4,2010Q1,Países Nórdicos,CANARIAS,537221.0,1193.58,448.43,294.31,450.84,14060.0,15458.0,...,259733.0,277487.0,527480.0,3717.0,2684.0,537220.0,3339.0,81875.0,455346.0,537220.0


# GUARDAMOS...

In [92]:
rutaDatosTratados = "../PredictorGastoTuristico/DatosGuardados/"
filtro_canarias.to_csv(rutaDatosTratados+"GMI_join_cc_filtro_canarias.csv", index = False, encoding='utf-8')